In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image
import keras
import os
import cv2

In [ ]:
root_dir = '/Volumes/tsanchen/carbs_estimate/density_library/double_burger/case_0/17_38_36_43EE39CF-9680-421C-A045-F12D646F8A99'
image_path =  os.path.join(root_dir, 'image.jpg')
peripheral_path = os.path.join(root_dir, 'peripheral.json')

model_path = '/Users/Frost/Desktop/model_checkpoint_06_2.hdf5'

UNIFIED_IMAGE_SIZE = (512, 512)

def center_crop(array):
    assert len(array.shape) >= 2
    if array.shape[0] == array.shape[1]:
        return array
    shape_difference = abs(array.shape[0] - array.shape[1])
    offset = shape_difference // 2
    if array.shape[0] > array.shape[1]:
        return array[offset:array.shape[1] + offset, :]
    else:
        return array[:, offset:array.shape[0] + offset]

def center_normalize(image):
    mean = np.mean(cv2.resize(image, (512, 512)), axis=(0, 1))
    std = np.std(cv2.resize(image, (512, 512)), axis=(0, 1))
    return (image - mean) / std

In [ ]:
model = keras.models.load_model(model_path)

In [ ]:
with open(peripheral_path) as in_file:
    json_content = json.loads(in_file.read())
    depth_map = np.array(json_content['depth_data'])
image = np.array(Image.open(image_path))
plt.imshow(image)

In [ ]:
plt.imshow(depth_map)
plt.colorbar()

In [ ]:
mask = np.reshape(
    model.predict(
        np.reshape(
            center_normalize(
                cv2.resize(
                    center_crop(
                        image
                    ), 
                    UNIFIED_IMAGE_SIZE
                )
            ), 
            (1, *UNIFIED_IMAGE_SIZE, 3)
        )
    ), UNIFIED_IMAGE_SIZE
)
plt.imshow(mask)
plt.colorbar()